In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install fsspec
!pip install gcsfs        
!pip install git+https://github.com/qubvel/segmentation_models
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-dl-car-classification/train.zip
/kaggle/input/sf-dl-car-classification/sample-submission.csv
/kaggle/input/sf-dl-car-classification/test.zip
/kaggle/input/sf-dl-car-classification/train.csv
  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-4qzagkm0
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-4qzagkm0
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 263 kB/s eta 0:00:01
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33792 sha256=fb35b1caf1d36ca7e90026484f6dc68c08c14d3d26f04e566aabaffcabe392a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-8dr8oj_c/wheels/02/cd/18/61c0bbb8766acfec68f9d20618886b7b38dfeeb95865b6ba00
Successfully built segmentation-models


In [2]:
!pip install tensorflow --upgrade #tensorflow

!pip install -q efficientnet# pre-trained net

!pip install git+https://github.com/mjkvaak/ImageDataAugmentor #augmentation

ERROR: Invalid requirement: 'efficientnet#'
  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-abdgqt2a
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-abdgqt2a
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29531 sha256=78bb7a9048b8efc79ecc4c55c69086a37d086cdae5ca168d5b1e8c6ffcafa76e
  Stored in directory: /tmp/pip-ephem-wheel-cache-c0cj89pl/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
Successfully built ImageDataAugmentor


# How to train your:
## {-- Car Identification Algorithm --}
### (with keras, efficentnetB6, and augmentation)
<img src="https://i.postimg.cc/449Smnsv/download.jpg" width="750px"/>


# Goal:
to train a new head on EfficientnetB6 to optimize for our problem utilizing augmentation

# Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import efficientnet.tfkeras as efn

from tensorflow.keras.layers import *

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter

from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

%config InlineBackend.figure_format = 'svg' 
%matplotlib inline



## Hyperparameters

In [4]:
EPOCHS               = 10  # training epochs
BATCH_SIZE           = 32 # optimize batches for GPU
LR                   = 1e-3 # learning rate
VAL_SPLIT            = 0.2 # test size

CLASS_NUM            = 10  # 10 car classes
IMG_SIZE             = 224 # image size 224x224
IMG_CHANNELS         = 3   # RGB
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)


DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/cars/"



RANDOM_SEED = 69
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA

In [5]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
print(train_df.info(),"\n", "="*20)
print(train_df.head())
print("\n", "="*20, f"\nCategories: {train_df.Category.nunique()} \n", train_df.Category.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15561 entries, 0 to 15560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        15561 non-null  object
 1   Category  15561 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 243.3+ KB
None 
           Id  Category
0  100155.jpg         0
1  100306.jpg         0
2  100379.jpg         0
3  100380.jpg         0
4  100389.jpg         0

Categories: 10 
 1    1971
8    1765
6    1733
5    1631
0    1613
3    1528
2    1458
4    1400
9    1255
7    1207
Name: Category, dtype: int64


In [6]:
print('Unloading pictures...\n')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as file:
        file.extractall(PATH)
        
print(os.listdir(PATH))

Unloading pictures...

['test_upload', 'train']


In [ ]:
# Let's look at the images
plt.figure(figsize=(12,8))

image = train_df.sample(n=9)
img_paths = image['Id'].values
img_cat = image['Category'].values

for index, path in enumerate(img_paths):
    im = PIL.Image.open(PATH+f'train/{img_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(img_cat[index]))
    plt.axis('off')
plt.show()

Like shoping on avito...


In [ ]:
import random
folder = './cars/train/5'
a=random.choice(os.listdir(folder))


image = PIL.Image.open(folder+'/'+a)
imgplot = plt.imshow(image)
plt.show()
print("Image size: ",image.size)

# Augmentation

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose)


p=0.9

AUGMENTATIONS = Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

In [ ]:
# apply augmentations
from ImageDataAugmentor.image_data_augmentor import *
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )
        
test_datagen = ImageDataAugmentor(rescale=1./255)

In [ ]:
# Generate images
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',     
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

# Modeling

## Base model EfficientnetB6

In [ ]:
base_model = efn.EfficientNetB6(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# Let's freeze the model and only train the head
base_model.trainable = False

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as C
from tensorflow.keras.preprocessing import image


model=M.Sequential() # Initialize

model.add(base_model) #base

#the Head v
model.add(L.GlobalAveragePooling2D(),)
model.add(L.Dense(256, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.25))
model.add(L.Dense(CLASS_NUM, activation='softmax'))

## Training the model

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

checkpoint = ModelCheckpoint('best_yet.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop] # callbacks for the best model and early stop if the performance doesn't improve

In [ ]:
#training
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('../working/model_last.hdf5')
model.load_weights('best_yet.hdf5')

## Tuning

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
print("Number of trainable variables", len(base_model.trainable_variables))

In [ ]:
EPOCHS               = 10
BATCH_SIZE           = 16 # decreasing batch size
LR                   = 1e-4 #increasing LR to fine tune even finer

In [ ]:
# Same as before
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',    
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

  
checkpoint = ModelCheckpoint('best_yet.hdf5' , monitor = ['accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

# Training
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('../working/model_last.hdf5')
model.load_weights('best_yet.hdf5')

Now unfreeze the whole model

In [ ]:
base_model.trainable = True
EPOCHS               = 8  
BATCH_SIZE           = 8 #even smaller
LR                   = 1e-5 # even larger

In [ ]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',     
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

  
checkpoint = ModelCheckpoint('best_yet.hdf5' , monitor = ['accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

#Training

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [ ]:
scores = model.evaluate_generator(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('../working/model_last.hdf5')
model.load_weights('best_yet.hdf5')

# Prediction on Test

In [ ]:
from sklearn.metrics import accuracy_score
test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)


In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [13]:
import shutil
shutil.rmtree("../working/cars")

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')